# Black-Scholes Option Pricing Model
## Sector: Energy <br />Industry: Oil & Gas Refining and Marketing

In [17]:
import warnings
warnings.simplefilter(action='ignore')
import pandas as pd
from scipy.stats import norm
import math
from math import e
from numpy import log as ln
from datetime import datetime

df = pd.read_csv('oil.csv', low_memory=False)
spots = pd.read_csv('oil_spot.csv', low_memory=False)
df['strike_price'] = df['strike_price']/1000
df = df[['date','exdate','cp_flag','strike_price','best_bid','best_offer','impl_volatility','optionid','ticker']]
groups = df.groupby('ticker')
spot_groups = spots.groupby('ticker')
tickers = df.drop_duplicates('ticker')
tickers = tickers['ticker'].to_list()

In [18]:
data = groups.get_group('BPT')
sdata = spot_groups.get_group('BPT')
data = data.merge(sdata,on='date')
data = data.sort_values(by=['date','exdate']).drop(columns='ticker_y').rename(columns={'ticker_x':'ticker'}).dropna().drop_duplicates('optionid').reset_index()
data1 = data.groupby('cp_flag')
data = data1.get_group('C')

In [19]:
bs = []
for x in data.iloc:
    date1 = datetime.strptime(x['date'], "%m/%d/%y")
    date2 = datetime.strptime(x['exdate'], "%m/%d/%y")

    C = x['best_offer']
    St = x['spot']
    K = x['strike_price']
    r = 0.03
    t = ((date2 - date1).days)/365
    o = x['impl_volatility']

    d1 = (ln(St/K)+(r+((o**2)/2))*t)/o*math.sqrt(t)
    d2 = d1-o*math.sqrt(t)

    price = norm.cdf(d1)*St - norm.cdf(d2)*K*e**(-r*t)
    bs.append(price)

data['black_scholes'] = bs
final = data[['ticker','date','exdate','strike_price','spot','cp_flag','best_bid','best_offer','black_scholes']]
final = final.reset_index().drop(columns=['index'])
#for x,y in zip(final.iloc, range(len(final))):
#    if x['black_scholes'] < 0:
#        final = final.drop(y,axis=0)
        
#final = final.reset_index().drop(columns=['level_0','index'])
final

,ticker,date,exdate,strike_price,spot,cp_flag,best_bid,best_offer,black_scholes
0,BPT,01/03/17,01/20/17,22.5,24.90,C,1.70,3.20,2.042840
1,BPT,01/03/17,01/20/17,25.0,24.90,C,0.45,0.85,0.826639
2,BPT,01/03/17,01/20/17,30.0,24.90,C,0.00,0.10,-0.967316
3,BPT,01/03/17,02/17/17,15.0,24.90,C,9.20,10.70,7.888965
4,BPT,01/03/17,02/17/17,22.5,24.90,C,2.00,3.20,2.646241
...,...,...,...,...,...,...,...,...,...
274,BPT,12/18/17,02/16/18,17.5,20.05,C,1.95,3.70,2.746401
275,BPT,12/18/17,02/16/18,20.0,20.05,C,0.25,0.70,0.673324
276,BPT,12/18/17,02/16/18,22.5,20.05,C,0.00,0.45,0.160062
277,BPT,12/18/17,02/16/18,25.0,20.05,C,0.00,0.35,-0.279582
